# Linear Regression

This notebook demonstrates how to use `rice_ml.supervised_learning.linear_regression.LinearRegression`
to fit a regression model on a real dataset.

We will:
1. Load a standard regression dataset (Diabetes).
2. Standardize features (important for gradient descent stability).
3. Fit OLS and Ridge models (closed-form solver).
4. Optionally fit a Gradient Descent (GD) version and visualize the loss curve.
5. Evaluate results using RMSE and R², and visualize predictions.



In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

def add_repo_src_to_path(max_up: int = 8) -> None:
    cur = os.path.abspath(os.getcwd())
    for _ in range(max_up):
        candidate = os.path.join(cur, "src")
        if os.path.isdir(os.path.join(candidate, "rice_ml")):
            if candidate not in sys.path:
                sys.path.insert(0, candidate)
            return
        cur = os.path.abspath(os.path.join(cur, ".."))
    raise RuntimeError("Could not find 'src/rice_ml'. Run this notebook inside the repo, or install the package.")

add_repo_src_to_path()

from rice_ml.processing.preprocessing import standardize
from rice_ml.supervised_learning.linear_regression import LinearRegression

np.random.seed(42)


## 2. Load a regression dataset

We use the Diabetes dataset (10 features, target is a quantitative disease progression measure).
This dataset is small but realistic, and is commonly used for regression demos.


In [ ]:
try:
    from sklearn.datasets import load_diabetes
    from sklearn.model_selection import train_test_split
except Exception as e:
    raise ImportError("This notebook needs scikit-learn installed (for datasets/splitting).") from e

data = load_diabetes()
X = data.data
y = data.target
feature_names = list(data.feature_names)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("First 5 feature names:", feature_names[:5])


## 3. Train/test split and standardization

We standardize features using training statistics only (mean/std from train),
then apply the same transform to the test split. This is especially important
for gradient descent and for comparing coefficients across features.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

X_train_std, params = standardize(X_train, return_params=True)

# Be robust to how `standardize` stores params (function signature may differ between versions).
import inspect
sig = inspect.signature(standardize)
if "params" in sig.parameters:
    X_test_std = standardize(X_test, params=params)
else:
    # common fallback: params is (mean, std) or dict with keys
    if isinstance(params, dict):
        mean = params.get("mean")
        std = params.get("std")
    else:
        mean, std = params
    X_test_std = (X_test - mean) / std

print("Train mean (approx):", X_train_std.mean(axis=0)[:5])
print("Train std (approx):", X_train_std.std(axis=0)[:5])
